In [1]:
import os
import sys
app_path = os.path.abspath('..')
sys.path.insert(0, app_path)

from datetime import datetime, UTC

from langchain import PromptTemplate
from langchain_ollama import ChatOllama

from app.chroma_client import get_embeddings, get_client

In [2]:
channel_name = 'go_to_vilnius'
channel_id = -1133953167

In [3]:
# ru_model_name = "cointegrated/rubert-tiny2"
ru_model_name = "intfloat/multilingual-e5-large"
embeddings = get_embeddings(ru_model_name)

chroma_client_from_telegram = get_client(f'telegram_{channel_name}', embeddings)

In [4]:
llm_qwen3_8b = ChatOllama(
    model="qwen3:8b",
)

In [5]:
user_query = "Выведи самые токсичные сообщения в чате."
user_query = "Выведи самые позитивные сообщения в чате."
user_query = "Выведи самые негативные сообщения с Литве и Вильнюсе в чате."
# user_query = "Можно ли компенсировать отель или аквапарк через велнес/БТА/бта/BTA?"
# user_query = "Как получить детские деньги или компенсацию по уходу за ребенком."

In [6]:
ru_telegram_prompt_expanding = PromptTemplate(
    input_variables=["user_query", "n"],
    template="""
    Напиши {n} гипотетических ответов на запрос пользователя.
    Не добавляй заголовков, авторов и другой дополнительной информации к твоим ответам.

    Вопрос пользователя:
    {user_query}

    Гипотетические ответы:
    """,
)

ru_telegram_prompt_expanding = PromptTemplate(
    input_variables=["user_query", "n"],
    template="""
Контекст данных:
  – Пользовательский запрос: {user_query}

Задача:
  Сгенерировать ровно {n} гипотетических, но практически релевантных ответов на основе вышеуказанных сообщений.

Требования к каждому ответу:
  1. Длина не более 2–3 предложений.
  2. Строгая релевантность: использовать только смысловую информацию из сообщений, без «галлюцинаций».
  4. Формат выходных данных: маркированный.
  5. Не добавлять заголовки, авторов, ссылки или другие метаданные.

Пример:
  Вопрос: «Какие самые позитивные сообщения в чате?»
  Ответ:
  - Ого, какая замечательная идея! Это точно поднимет настроение всем!
  - Спасибо за вашу помощь — вы просто спасли мой день!
  - Мне очень нравится, как вы подходите к решению задач: вдохновляет!
  - Ваше сообщение заставило меня улыбнуться — вы лучик света в этом чате!
  - Благодарю за поддержку, с вами так легко общаться!
""",
)

hyde_chain_r1_8b = ru_telegram_prompt_expanding | llm_qwen3_8b
raw_hypothetical_answers = hyde_chain_r1_8b.invoke({"user_query": user_query, "n": 15}).content
raw_hypothetical_answers

ResponseError: llama runner process has terminated: error loading model: unable to allocate CUDA0 buffer
llama_model_load_from_file_impl: failed to load model (status code: 500)

In [11]:
hypothetical_answers = raw_hypothetical_answers.split('</think>', maxsplit=1)[-1]
user_query_expanded = f'{user_query}\n\n{hypothetical_answers}'

In [12]:
related_docs = chroma_client_from_telegram.similarity_search_with_relevance_scores(user_query_expanded, k=15)
related_docs

[(Document(id='6244d880-9a47-47e6-ad6c-e1cdf1515659', metadata={'date': 1673898460.0, 'id': 79574, 'sender_name': 'stivo32'}, page_content='По приезде в Вильнюс вы можете получить сим-карту. Она выдаётся в отеле. Сотрудник самостоятельно может сменить тариф при необходимости и пополнить баланс. \nДелается это (смена тарифа или пополнение баланса) на сайте оператора или в мобильном приложении.\n\nПредоплаченные сим-карты с планами за передачу данных, звонков и СМС стоят 3–10 евро. Сим-карты доступны в газетных киосках Narvesen и Lietuvos Spauda, а также в большинстве крупных супермаркетов на кассах. \n\nОператоры мобильной связи в Литве:\nLabas - https://www.labas.lt/\nPildyk - https://pildyk.lt/\nEžys - https://www.ezys.lt/\n(чем они отличаются пока загадка, одинаково хорошее покрытие, странные сайты и тарифы)\n❗ тут нет ерипа, пополнить счёт можно след способами:\n- с сайта оператора\n- с приложения оператора\n- в магазине на кассе\n\nИнтернет-провайдеры в Литве:\nI would like to shar

In [23]:
from datetime import datetime, timedelta, UTC

related_docs = chroma_client_from_telegram.get(
    where={"date": {"$gte": (datetime.now(UTC) - timedelta(days=30)).timestamp()}},
)
meta_docs = list(zip(related_docs['metadatas'], related_docs['documents']))

context = "\n\n---\n\n".join(f"{meta['date']} - {meta['sender_name']}: {doc}" for meta, doc in meta_docs)

In [25]:
len(related_docs['metadatas'])

805

In [40]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

t_link_base = "https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D"

print("\n\n---\n\n".join(f"""
{datetime.fromtimestamp(doc.metadata['date'], UTC)} {t_link_base}{doc.metadata['id']} - {doc.page_content}
""" for doc, _score in filtered_related_docs))


2024-08-23 14:22:40+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D108686 - Коллеги, а кто нибудь пробовал с бта велнеса компенсить отель со спа? Прокатывает?


---


2024-08-23 14:25:55+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D108687 - >> Коллеги, а кто нибудь пробовал с бта велнеса компенсить отель со спа? Прокатывает?

 не знаю, как отель со спа, но баню компенсил, прокатывало)


---


2025-03-13 13:58:29+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D113452 - >> Ребята, привет!  А кто то компенсил в БТА Membership? На спа/басик? Разовые то точно компенсят, а если я возьму мембер на 5 походов, закомпенсят?

 Мне абонемент в бассейн компенсировали полностью


---


2024-05-10 14:33:38+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D105391 - А какая сейчас обстакановка со всякими ма

In [13]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

context = "\n\n---\n\n".join(f"{datetime.fromtimestamp(doc.metadata['date'], UTC)} - {doc.page_content}" for doc, _score in filtered_related_docs)

In [14]:
ru_telegram_prompt = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""
    Ты полезный AI ассистент, который отвечает на вопросы пользователя на основе контекста.
    Контекст это релевантные вопросу сообщения из телеграм чата.

    Контекст:
    {context}

    Вопрос пользователя:
    {user_query}

    Если в контексте недостаточно информации, чтобы ответить на вопрос пользователя, то скажи, что недостаточно информации.

    Ответ:
    """,
)

hyde_chain_r1_8b = ru_telegram_prompt | llm_qwen3_8b
llm_response = hyde_chain_r1_8b.invoke({"user_query": user_query, "context": context}).content
print(llm_response)

<think>
Okay, let's tackle this query. The user wants the most negative messages about Lithuania and Vilnius from the chat. First, I need to go through all the provided messages and identify the negative sentiments.

Looking at the messages, there are several points. For example, one user mentioned that in Vilnius, even puddles freeze in winter, which is pretty harsh. Another talked about rising prices in Lithuania outpacing salaries. There's also a mention of poor service quality for tourists, inefficient public transport, and bureaucratic challenges. Some users pointed out issues with internet quality and availability, especially in Vilnius. Others mentioned the lack of modern infrastructure and the difficulty in finding places for tourists. High taxes and low wages are also noted. 

I should list these points, making sure to capture the most negative aspects. I need to check if all the points are indeed negative and not just neutral observations. For example, the part about internet